In [50]:
"""
Created on 24-April-2024
@author: jdmantillaq
"""

import plotly.express as px
import pandas as pd
import numpy as np
import dash
from dash import Dash, dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input, State
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template
import os
import requests
import dash_leaflet as dl

In [57]:
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
app.title = 'Cambio Climático'

load_figure_template("darkly")
slider_style = {"font-size": "18px"}
text_style = {"font-size": "20px"}


dropdown = dbc.DropdownMenu(
    label="Menu",
    children=[
        dbc.DropdownMenuItem("Item 1"),
        dbc.DropdownMenuItem("Item 2"),
        dbc.DropdownMenuItem("Item 3"),
    ], direction="end"
)

map = dl.Map([dl.TileLayer(),
              dl.FullScreenControl(),
              dl.ScaleControl(position="bottomleft")],
             zoom=4.5, center=(6.248011, -75.577277),
             style={'width': '100%',
                    'height': '75vh',
                    'margin': "auto",
                    "display": "block"})


models_NEX = ['ACCESS-CM2', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CanESM5',
          'CMCC-CM2-SR5', 'CMCC-ESM2', 'EC-Earth3',
          'EC-Earth3-Veg-LR', 'GFDL-CM4',
          'GFDL-ESM4', 'INM-CM4-8', 'INM-CM5-0',
          'IPSL-CM6A-LR', 'KACE-1-0-G', 'KIOST-ESM', 'MIROC6',
          'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NESM3',
          'NorESM2-LM', 'NorESM2-MM', 'TaiESM1']


models_HighResp = ['FGOALS-f3-L', 'MRI-AGCM3-2-H',
          'MRI-AGCM3-2-S', 'NICAM16-7S', 'NICAM16-8S']

obs_list = ['ERA5', 'CHRIPS']

radioitems_dataset_obs = dbc.CardBody(dcc.RadioItems(obs_list,
                                                     id="radio_dataset_obs",),
                                      style=slider_style)


radioitems_dataset_NEX = dbc.CardBody(dcc.RadioItems(models_NEX,
                                                     id="radio_dataset_NEX",),
                                      style=slider_style)

radioitems_dataset_HighResp = dbc.CardBody(dcc.RadioItems(models_HighResp,
                                                          id="radio_dataset_HighResp",),
                                      style=slider_style)



content_dataset = html.Div(
    dbc.Row(
        [dbc.Col([html.H5(children="Observaciones"),
                  html.Hr(),
                  radioitems_dataset_obs], width=4),
         dbc.Col([html.H5(children="NEX-GDDP-CMIP6"),
                  html.Hr(),
                  radioitems_dataset_NEX], width=4),
         dbc.Col([html.H5(children="HighResMIP-CMIP6"),
                  html.Hr(),
                  radioitems_dataset_HighResp], width=4),]
    )

)




popovers_dataset = html.Div(
    [
        dbc.Button(
            "Dataset",
            id="button_dataset",
            n_clicks=0,
            style=slider_style
        ),
        dbc.Popover(
            [
                # dbc.PopoverHeader("Popover header"),
                dbc.PopoverBody(content_dataset),
            ],
            target="button_dataset",
            placement='bottom',
            trigger="legacy",
            autohide=True,
            style={'maxWidth': '2000px'},
            # body=True,
        ),
    ]
)


popovers_var = html.Div(
    [
        dbc.Button(
            "Variable/Index",
            id="button_var",
            n_clicks=0,
        ),
        dbc.Popover(
            [
                # dbc.PopoverHeader("Popover header"),
                dbc.PopoverBody(dropdown),
            ],
            target="button_var",
            placement='bottom',
            trigger="legacy",
        ),
    ]
)


tab1_content = dbc.Card(
    dbc.CardBody(
        [dbc.Row([dbc.Col([popovers_dataset],
                          style={"display": "flex", "flexWrap": "wrap"})]),
         dbc.Row(map)]
    ))


tab2_content = dbc.Card()


app.layout = html.Div(
    [html.Div(
        className="header",
        children=[
            html.Div(
                className="div-info",
                children=[
                    html.H3(className="title",
                            children="Cambio Climático",
                            style={"padding": "10px"}),
                    html.Hr(),
                    dbc.Tabs([
                        dbc.Tab(tab1_content, label="Índices de Extremos",
                                tab_id="tab-1")],
                             id="tabs",
                             active_tab="tab-1",
                             )
                ],
            ),
        ],
    ),
    ], style=text_style
)


@app.callback(
    [Input("radio_dataset_obs", "value"),
     Input("radio_dataset_NEX", "value"),
     Input("radio_dataset_HighResp", "value")],
    [Input("radio_dataset_obs", "value"),
     Input("radio_dataset_NEX", "value"),
     Input("radio_dataset_HighResp", "value")]
)
def clear_other_selections(obs_value, nex_value, high_value):
    print(obs_value, nex_value, high_value)
    if obs_value is not None:
        return obs_value, None, None
    elif nex_value is not None:
        return None, nex_value, None
    else:
        return None, None, high_value


app.run_server(port=2100, host='0.0.0.0')